# Lab 5.4: Chi-Square Tests

## Outline

- Chi-square test for goodness of fit
- Chi-square test for independence

**Questions 1**  

Seven percent of mutual fund investors rate corporate stocks “very safe,” 58% rate them “somewhat safe,” 24% rate them “not very safe,” 4% rate them “not at all safe,” and 7% are “not sure.” A BusinessWeek/Harris poll asked 529 mutual fund investors how they would rate corporate bonds on safety. The responses are as follows.  

<img src="images/q1.png" width="300">  

Do mutual fund investors’ attitudes toward corporate bonds differ from their attitudes toward
corporate stocks? Clearly state the null and alternative hypotheses.

In [1]:
%pylab inline

import pandas as pd
import statsmodels.api as sm

from IPython.display import Latex

from scipy import stats
from statsmodels.graphics.gofplots import qqplot
from statsmodels.distributions.empirical_distribution import ECDF

Populating the interactive namespace from numpy and matplotlib


In [2]:
saftey = ["verysafe", "somewhatsafe", "notverysafe", "notatallsafe", "notsure"]
data = pd.DataFrame({"Stocks": [0.07,0.58, 0.24, 0.04, 0.07], "Bonds": [48/529, 323/529, 79/529, 16/529, 63/529]}, index=saftey).T
data

,verysafe,somewhatsafe,notverysafe,notatallsafe,notsure
Bonds,0.090737,0.610586,0.149338,0.030246,0.119093
Stocks,0.070000,0.580000,0.240000,0.040000,0.070000


$H_0$: the stocks and bonds attitudes are same  

$H_a$: the stocks and bonds attitudes differ

In [3]:
n = 529
# expected proportions
p_stocks = np.array([.07, .58, .24, .04, .07])
# expected counts
e_bonds = n*p_stocks
# observed counts
o_bonds = np.array([48, 323, 79, 16, 63])
# calculate the test statistic
chi_2 = np.sum(((o_bonds - e_bonds)**2)/e_bonds)
print('chi^2:', chi_2)
# calculate p-value
stats.chi2(4).sf(chi_2)

chi^2: 41.6919444005


1.9323272579221198e-08

The p-value is less than .05, so we reject the null hypothesis that attitudes towards stocks and bonds are equivalent.

**Question 2**  

A news article reports that "Americans have differing views on two potentially inconvenient and invasive practices that airports could implement to uncover potential terrorist attacks." This news piece was based on a survey conducted among a random sample of 1,137 adults nationwide, interviewed by telephone November 7-10, 2010, where one of the questions on the survey was "Some airports are now using 'full-body' digital x-ray machines to electronically screen passengers in airport security lines. Do you think these new x-ray machines should or should not be used at airports?" Below is a summary of responses based on party affiliation.  

<img src="images/q4.png" width="550">  

The differences in each political group may be due to chance. Complete the following computations under the null
hypothesis of independence between an individual's party affiliation and his/her support of full-body scans. It may be useful to first add on an extra column for row totals before proceeding with the computations.  

1) How many Republicans would you expect to not support the use of full-body scans?  

2) How many Democrats would you expect to support the use of full-body scans?  

3) How many Independents would you expect to not know or not answer?  

4) Test if an individual's party affiliation affects his/her support of full-body scans. Clearly state the null and alternative hypotheses in your test.

In [4]:
answer = ["should", "shouldnot", "dontknow"]
party = pd.DataFrame({"republican": [264,38,16], 
                      "democrat": [299,55,15], 
                      "independent": [351,77,22]}, index=answer).T
party_chi2, p, dof, party_expected = stats.chi2_contingency(party.T)
party.T

,democrat,independent,republican
should,299,351,264
shouldnot,55,77,38
dontknow,15,22,16


In [5]:
party_expected

array([[ 296.62796834,  361.7414248 ,  255.63060686],
       [  55.17150396,   67.2823219 ,   47.54617414],
       [  17.2005277 ,   20.9762533 ,   14.823219  ]])

In [6]:
expected_df = pd.DataFrame(party_expected, index=['democrat', 'independent', 'republican'], columns=answer)

print("\nObserved:\n{}".format(party))
print("\nExpected:\n{}".format(expected_df))


Observed:
             should  shouldnot  dontknow
democrat        299         55        15
independent     351         77        22
republican      264         38        16

Expected:
                 should   shouldnot    dontknow
democrat     296.627968  361.741425  255.630607
independent   55.171504   67.282322   47.546174
republican    17.200528   20.976253   14.823219


#### 1) How many Republicans would you expect to not support the use of full-body scans?


In [7]:
print("We would expect", round(party_expected[2][1], 2), "Republicans to not support body scans")

We would expect 20.98 Republicans to not support body scans


#### 2) How many Democrats would you expect to support the use of full-body scans?


In [8]:
print("We would expect", round(party_expected[0][0], 2), "Democrats to support body scans")

We would expect 296.63 Democrats to support body scans


#### 3) How many Independents would you expect to not know or not answer?


In [9]:
print("We would expect", round(party_expected[1][2], 2), "Independents to not not know/not answer about body scans")

We would expect 47.55 Independents to not not know/not answer about body scans


#### 4) Test if an individual's party affiliation affects his/her support of full-body scans. Clearly state the null and alternative hypotheses in your test.

In [10]:
Latex(r"$\chi^2 = {:.4}; p = {:.2}$".format(party_chi2, p))

<IPython.core.display.Latex object>

$H_0$: party affiliation does not affect support of full body scan (party affiliation and support independent)

$H_a$: party affiliation affects support of full body scan (party affiliation and support not independent)  

Since p-value(0.36 > .05), we fail to reject the null hypothesis. We don't have enough evidence that party affiliation affects support of the full body scan issue

**Question 3**  

A clothes retailer believes that there is no difference in sales across Monday, Tuesday and Wednesday. You are given the data in a `cloth_sales` table (in RDS) to test the claim. The table contains two columns: `dt` for the date, and `sales`, containing the count of sales for that day. Start by drawing up the table for the observed and expected frequencies for the chi-square test. 

**Hint:** 
- It will probably be easiest to extract the week of the year (`week`) and the day of the week (`dow`) using the [`date_trunc`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) and [`date_part`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions in PostgreSQL respectively. You can then [`pivot`](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-by-pivoting-dataframe-objects) the table with pandas. The `head` of the resulting data frame should look something like:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>date_part</th>
      <th>1.0</th>
      <th>2.0</th>
      <th>3.0</th>
    </tr>
    <tr>
      <th>date_trunc</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2014-10-27</th>
      <td>1016</td>
      <td>978</td>
      <td>1010</td>
    </tr>
    <tr>
      <th>2014-11-03</th>
      <td>987</td>
      <td>991</td>
      <td>997</td>
    </tr>
    <tr>
      <th>2014-11-10</th>
      <td>1014</td>
      <td>983</td>
      <td>1002</td>
    </tr>
    <tr>
      <th>2014-11-17</th>
      <td>991</td>
      <td>945</td>
      <td>992</td>
    </tr>
    <tr>
      <th>2014-11-24</th>
      <td>1001</td>
      <td>1058</td>
      <td>1002</td>
    </tr>
  </tbody>
</table>  

(N.B. `date_part('dow', dt)` will return the number of days after Sunday, so Monday = 1.0, Tuesday = 2.0, and so on.)
   
- Use `scipy.stats.chisquare()` to carry out a goodness of fit test

In [11]:
import yaml

from sqlalchemy import create_engine

pg_creds = yaml.load(open('../../pg_creds.yaml'))['student']

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**pg_creds))

In [12]:
%load_ext sql

/Users/srianant/anaconda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/srianant/anaconda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [13]:
# create pandas df from SQL
sales = pd.read_sql("SELECT date_trunc('week', cloth_sales.dt) as week_of_year, date_part('dow', cloth_sales.dt) as day_of_week, sales from cloth_sales", engine)
sales.head()

,week_of_year,day_of_week,sales
0,2014-10-27,1.0,1016
1,2014-10-27,2.0,978
2,2014-10-27,3.0,1010
3,2014-11-03,1.0,987
4,2014-11-03,2.0,991


In [16]:
# create pivot table
sales = sales.pivot('week_of_year', 'day_of_week')
sales.head()

sales            
day_of_week    1.0   2.0   3.0
week_of_year                  
2014-10-27    1016   978  1010
2014-11-03     987   991   997
2014-11-10    1014   983  1002
2014-11-17     991   945   992
2014-11-24    1001  1058  1002

In [17]:
# calculate expected counts for each week
sales['total'] = sales[['sales'][0]][1.0] + sales[['sales'][0]][2.0] + sales[['sales'][0]][3.0]
sales['expected'] = sales['total']/3
sales.head()

sales             total     expected
day_of_week    1.0   2.0   3.0                   
week_of_year                                     
2014-10-27    1016   978  1010  3004  1001.333333
2014-11-03     987   991   997  2975   991.666667
2014-11-10    1014   983  1002  2999   999.666667
2014-11-17     991   945   992  2928   976.000000
2014-11-24    1001  1058  1002  3061  1020.333333

In [21]:
# calculate X^2 statistic
x_2 = 0
for day in [1.0, 2.0, 3.0]:
    x_2 = np.sum(((sales[['sales'][0]][day] - sales['expected'])**2)/sales['expected'])
x_2

24.697211952470848

In [22]:
# calculate p-value
df=2
stats.chi2(2).sf(x_2)

4.3357932540058771e-06

Since p > .05, we reject the null hypothesis, which is that there is no difference in sales across days of the week.

**Question 4**  

1) A law suit has been filed against a university for a charge of sexual discrimination against female applicants during the admissions process. Use the data below to test whether sex affects admission at this university.
      
   **Hint:**
   - Construct your null and alternative hypotheses
   - Use `scipy.stats.chi2_contingency()` to carry out a test for independence
   - The function takes the contingency table as an `numpy` array as the first argument


|        | Admitted | Not Admitted |
|--------|----------|--------------|
| Male   | 3715     | 4727         |
| Female | 1513     | 2808         |

2) You are also given the breakdown of the female and male admissions by department (A to F).

<img src="images/paradox_1.png" width="300px">

Test if sex and department are independent in terms of number of applicants.

3) (Extra Credit) Based on all the data you are given, is it fair to say that there is sexual discirmination in the admission process? Explain your answer. (Hint: Simpson's paradox)

#### Solution 4.1

In [23]:
admin = ["Admitted", "NotAdmitted"]
sdata = pd.DataFrame({"Male": [3715,4727], "Female": [1513,2808]}, index=admin).T
sdata

,Admitted,NotAdmitted
Female,1513,2808
Male,3715,4727


$H_0$: admission & gender are independent
    
$H_a$: admission & gender not independent

In [24]:
chi2, p, dof, expected = stats.chi2_contingency(sdata)

Latex(r"$\chi^2 = {:.4}; p = {:.2}$".format(chi2, p))

<IPython.core.display.Latex object>

#### 4.1 commentary

Since p-value < .05 (1.7e−22) is extremly small, we reject NULL-hypothesis. Not enough evidence to say that admission rates & gender are independent.

#### Solution 4.2

$H_0$: department & gender are independent
    
$H_a$: department & gender are not independent

In [29]:
applicants = ["MalesAdmitted","FemalesAdmitted"]
department = pd.DataFrame({"A": [825,108], "B": [560,25], "C": [325,593], "D": [417,375], "E": [191,393], "F": [373,341]}, index=applicants).T

department

,MalesAdmitted,FemalesAdmitted
A,825,108
B,560,25
C,325,593
D,417,375
E,191,393
F,373,341


In [30]:
print("MalesAdmitted: ",511.50+352.80+120.25+137.61+53.48+22.38)

MalesAdmitted:  1198.02


In [31]:
print("FemalesAdmitted: ",88.56+17.00+201.62+131.25+94.32+23.87)

FemalesAdmitted:  556.62


In [32]:
chi2, p, dof, expected = stats.chi2_contingency(department)

Latex(r"$\chi^2 = {:.4}; p = {:.2}$".format(chi2, p))

<IPython.core.display.Latex object>

#### 4.2 commentary

Since p-value < .05 (9.4e−229) is extremly small, we reject NULL-hypothesis. Not enough evidence to say that department & gender are independent.

#### 4.3 Commentary

No, despite the fact that both tests showed us that we have to reject the null hypotheses that gender vs. department and gender vs. admissions are independent, we can't definitively say that sexual discrimination is happening. Gender vs. admissions shows bias toward the male direction while gender vs. department shows bias towards the female direction. Two biases combined together can create misleading results/conclusions, similar to what Anscombe's Quartet demonstrates. 

#### Hint

Some functions that may be useful to you:

- From the `sqlalchemy` package:
    - `create_engine`
- From the `pandas` package:
    - `read_sql`
    - `pivot`
- From the `scipy.stats` package:
    - `chisquare`
    - `chi2_contingency`